In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils;
#reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


## Setup

In [3]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [5]:
chars.insert(0, "\0")

In [6]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

Map from chars to indices and back again

In [10]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

*idx* will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [11]:
idx = [char_indices[c] for c in text]

In [12]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [13]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

## 3 char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [14]:
# replace range to xrange if required.
cs=3
c1_dat = [idx[i] for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

Our inputs

In [15]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

Our output

In [16]:
y = np.stack(c4_dat[:-2])

The first 4 inputs and outputs

In [17]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [18]:
y[:4]

array([30, 29,  1, 40])

In [19]:
x1.shape, y.shape

((200297,), (200297,))

The number of latent factors to create (i.e. the size of the embedding matrix)

In [20]:
n_fac = 42

Create inputs and embedding outputs for each of our 3 character inputs

In [21]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [22]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

### Create and train model

Pick a size for our hidden state

In [28]:
n_hidden = 256

In [29]:
dense_in = Dense(n_hidden, activation='relu')

In [30]:
c1_hidden = dense_in(c1)

In [31]:
dense_hidden = Dense(n_hidden, activation='tanh')

In [32]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])

In [33]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

In [34]:
dense_out = Dense(vocab_size, activation='softmax')

In [35]:
c4_out = dense_out(c3_hidden)

In [36]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [37]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [38]:
model.optimizer.lr=0.000001

In [39]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 42)         3612        c1[0][0]                         
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 42)         3612        c2[0][0]                         
___________________________________________________________________________________________

In [40]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 14s - loss: 4.4250    
Epoch 2/4
200297/200297 [==============================] - 14s - loss: 4.3148    
Epoch 3/4
200297/200297 [==============================] - 14s - loss: 4.0724    
Epoch 4/4
200297/200297 [==============================] - 14s - loss: 3.6849    


In [43]:
model.optimizer.lr=0.01

In [44]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 14s - loss: 3.3607    
Epoch 2/4
200297/200297 [==============================] - 14s - loss: 3.2179    
Epoch 3/4
200297/200297 [==============================] - 14s - loss: 3.1551    
Epoch 4/4
200297/200297 [==============================] - 14s - loss: 3.1248    


In [46]:
model.optimizer.lr = 0.000001

In [ ]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=20)

Epoch 1/20
200297/200297 [==============================] - 14s - loss: 2.7661    
Epoch 2/20
200297/200297 [==============================] - 14s - loss: 2.7582    
Epoch 3/20
200297/200297 [==============================] - 14s - loss: 2.7504    
Epoch 4/20
200297/200297 [==============================] - 14s - loss: 2.7428    
Epoch 5/20
 88576/200297 [============>.................] - ETA: 7s - loss: 2.7358

### Test model

In [53]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [65]:
get_next('phi')

't'

In [66]:
get_next(' th')

'e'

In [67]:
get_next(' an')

' '

## RNN

This is the size of our unrolled RNN.

In [73]:
cs=8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [74]:
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
            for n in range(cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [75]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [76]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [77]:
len(xs), xs[0].shape

(8, (75110,))

In [45]:
y = np.stack(c_out_dat[:-2])

So each column below is one series of 8 characters from the text.

In [78]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

...and this is the next character after each sequence.

In [57]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [58]:
n_fac = 42

### Create and train model

In [33]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [34]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [35]:
n_hidden = 256

In [36]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

The first character of each sequence goes through dense_in(), to create our first hidden activations.

In [37]:
hidden = dense_in(c_ins[0][1])

Then for each successive layer we combine the output of dense_in() on the next character with the output of dense_hidden() on the current hidden state, to create the new hidden state.

In [38]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

Putting the final hidden state through dense_out() gives us our output.

In [39]:
c_out = dense_out(hidden)

So now we can create our model.

In [179]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [180]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

Epoch 1/12
75110/75110 [==============================] - 3s - loss: 2.5385     
Epoch 2/12
75110/75110 [==============================] - 3s - loss: 2.2645     
Epoch 3/12
75110/75110 [==============================] - 3s - loss: 2.1596     
Epoch 4/12
75110/75110 [==============================] - 3s - loss: 2.0888     
Epoch 5/12
75110/75110 [==============================] - 3s - loss: 2.0355     
Epoch 6/12
75110/75110 [==============================] - 3s - loss: 1.9897     
Epoch 7/12
75110/75110 [==============================] - 3s - loss: 1.9506     
Epoch 8/12
75110/75110 [==============================] - 3s - loss: 1.9149     
Epoch 9/12
75110/75110 [==============================] - 3s - loss: 1.8840     
Epoch 10/12
75110/75110 [==============================] - 3s - loss: 1.8546     
Epoch 11/12
75110/75110 [==============================] - 3s - loss: 1.8293     
Epoch 12/12
75110/75110 [==============================] - 3s - loss: 1.8050     


### Test model

In [181]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [182]:
get_next('for thos')

'e'

In [432]:
get_next('part of ')

't'

In [433]:
get_next('queens a')

'n'

## RNN with Keras

In [30]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

This is nearly exactly equivalent to the RNN we built ourselves in the previous section.

In [31]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])

In [32]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_5 (Embedding)          (None, 8, 42)         3612        embedding_input_2[0][0]          
____________________________________________________________________________________________________
simplernn_2 (SimpleRNN)          (None, 256)           76544       embedding_5[0][0]                
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 86)            22102       simplernn_2[0][0]                
Total params: 102258
____________________________________________________________________________________________________


In [24]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [217]:
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 3s - loss: 2.7939     
Epoch 2/8
75110/75110 [==============================] - 3s - loss: 2.2970     
Epoch 3/8
75110/75110 [==============================] - 3s - loss: 2.0814     
Epoch 4/8
75110/75110 [==============================] - 3s - loss: 1.9416     
Epoch 5/8
75110/75110 [==============================] - 3s - loss: 1.8406     
Epoch 6/8
75110/75110 [==============================] - 3s - loss: 1.7625     
Epoch 7/8
75110/75110 [==============================] - 3s - loss: 1.6960     
Epoch 8/8
75110/75110 [==============================] - 3s - loss: 1.6421     


In [222]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [223]:
get_next_keras('this is ')

't'

In [224]:
get_next_keras('part of ')

't'

In [225]:
get_next_keras('queens a')

'n'

## Returning sequences

To use a sequence model, we can leave our input unchanged - but we have to change our output to a sequence (of course!)

Here, c_out_dat is identical to c_in_dat, but moved across 1 character.

In [64]:
#c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
#            for n in range(cs)]
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)]
            for n in range(cs)]

In [65]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

Reading down each column shows one set of inputs and outputs.

In [59]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [60]:
[ys[n][:cs] for n in range(cs)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

### Create and train model

In [47]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

We're going to pass a vector of all zeros as our starting point - here's our input layers for that:

In [48]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [66]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [67]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [68]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

(75110, 42)

In [394]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

INFO (theano.gof.compilelock): Refreshing lock /home/jhoward/.theano/compiledir_Linux-4.4--generic-x86_64-with-Ubuntu-16.04-xenial-x86_64-2.7.12-64/lock_dir/lock


Epoch 1/12
75110/75110 [==============================] - 7s - loss: 20.0841 - output_loss_1: 2.7123 - output_loss_2: 2.5681 - output_loss_3: 2.5143 - output_loss_4: 2.4739 - output_loss_5: 2.4675 - output_loss_6: 2.4442 - output_loss_7: 2.4627 - output_loss_8: 2.4410     
Epoch 2/12
75110/75110 [==============================] - 7s - loss: 17.8335 - output_loss_1: 2.5124 - output_loss_2: 2.3529 - output_loss_3: 2.2368 - output_loss_4: 2.1686 - output_loss_5: 2.1540 - output_loss_6: 2.1337 - output_loss_7: 2.1520 - output_loss_8: 2.1232     
Epoch 3/12
75110/75110 [==============================] - 7s - loss: 17.2340 - output_loss_1: 2.4967 - output_loss_2: 2.3306 - output_loss_3: 2.1766 - output_loss_4: 2.0814 - output_loss_5: 2.0529 - output_loss_6: 2.0291 - output_loss_7: 2.0475 - output_loss_8: 2.0192     
Epoch 4/12
75110/75110 [==============================] - 7s - loss: 16.8647 - output_loss_1: 2.4896 - output_loss_2: 2.3218 - output_loss_3: 2.1437 - output_loss_4: 2.0278 - out

### Test model

In [395]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [396]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'c', 's', ' ']

In [397]:
get_nexts(' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['t', 'o', 'r', 't', ' ', 'o', 'f', ' ']

### Sequence model with keras

In [50]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

To convert our previous keras model into a sequence model, simply add the 'return_sequences=True' parameter, and add TimeDistributed() around our dense layer.

In [67]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [52]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, 8, 42)         3612        embedding_input_3[0][0]          
____________________________________________________________________________________________________
simplernn_3 (SimpleRNN)          (None, 8, 256)        76544       embedding_6[0][0]                
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 8, 86)         22102       simplernn_3[0][0]                
Total params: 102258
____________________________________________________________________________________________________


In [71]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [82]:
xs[0].shape

(75110,)

In [90]:
x_rnn=np.stack(np.squeeze(xs), axis=1)
y_rnn=np.atleast_3d(np.stack(ys, axis=1))

In [91]:
x_rnn.shape, y_rnn.shape

((75110, 8), (75110, 8, 1))

In [92]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 4s - loss: 2.4284     
Epoch 2/8
75110/75110 [==============================] - 4s - loss: 2.0006     
Epoch 3/8
75110/75110 [==============================] - 4s - loss: 1.8863     
Epoch 4/8
75110/75110 [==============================] - 4s - loss: 1.8264     
Epoch 5/8
75110/75110 [==============================] - 4s - loss: 1.7882     
Epoch 6/8
75110/75110 [==============================] - 4s - loss: 1.7613     
Epoch 7/8
75110/75110 [==============================] - 4s - loss: 1.7417     
Epoch 8/8
75110/75110 [==============================] - 4s - loss: 1.7258     


In [93]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [94]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'i', 's', ' ']

## Stateful model with keras

In [290]:
bs=64

A stateful model is easy to create (just add "stateful=True") but harder to train. We had to add batchnorm and use LSTM to get reasonable results.

When using stateful in keras, you have to also add 'batch_input_shape' to the first layer, and fix the batch size there.

In [338]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [339]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

Since we're using a fixed batch shape, we have to ensure our inputs and outputs are a even multiple of the batch size.

In [340]:
mx = len(x_rnn)//bs*bs

In [341]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

INFO (theano.gof.compilelock): Refreshing lock /home/jhoward/.theano/compiledir_Linux-4.4--generic-x86_64-with-Ubuntu-16.04-xenial-x86_64-2.7.12-64/lock_dir/lock


Epoch 1/4
75072/75072 [==============================] - 13s - loss: 2.2051    
Epoch 2/4
75072/75072 [==============================] - 13s - loss: 1.9621    
Epoch 3/4
75072/75072 [==============================] - 13s - loss: 1.8893    
Epoch 4/4
75072/75072 [==============================] - 13s - loss: 1.8453    


In [342]:
model.optimizer.lr=1e-4

In [343]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 13s - loss: 1.8132    
Epoch 2/4
75072/75072 [==============================] - 13s - loss: 1.7877    
Epoch 3/4
75072/75072 [==============================] - 13s - loss: 1.7663    
Epoch 4/4
75072/75072 [==============================] - 13s - loss: 1.7475    


In [344]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 13s - loss: 1.7308    
Epoch 2/4
75072/75072 [==============================] - 13s - loss: 1.7155    
Epoch 3/4
75072/75072 [==============================] - 13s - loss: 1.7014    
Epoch 4/4
75072/75072 [==============================] - 13s - loss: 1.6881    


## Keras GRU

Identical to the last keras rnn, but a GRU!

In [101]:
model=Sequential([
        GRU(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [102]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 9s - loss: 2.3991     
Epoch 2/8
75110/75110 [==============================] - 9s - loss: 1.9818     
Epoch 3/8
75110/75110 [==============================] - 9s - loss: 1.8704     
Epoch 4/8
75110/75110 [==============================] - 9s - loss: 1.8070     
Epoch 5/8
75110/75110 [==============================] - 9s - loss: 1.7653     
Epoch 6/8
75110/75110 [==============================] - 10s - loss: 1.7346    
Epoch 7/8
75110/75110 [==============================] - 9s - loss: 1.7108     
Epoch 8/8
75110/75110 [==============================] - 9s - loss: 1.6918     


In [105]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'i', 's', ' ']